In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('SoFlaLiveLocal—Evergreen - Sheet1.csv')

In [5]:
df

,full_address,developers,description,Coordinates,story_link
0,Southwest 280th St./Waldin Drive between South...,"Poncecat Mandarin LLC, a joint venture between...",NaN,"25.50764286458881, -80.43624653378629",https://www.bizjournals.com/southflorida/news/...
1,"2110, 2118 and 2134 North Miami Avenue, as wel...",Bazbaz Development,"48-story, 544-unit partment tower","25.79729007306927, -80.19535522477574",NaN
2,2534 North Miami Avenue,Hidrock Properties and Robert Finvarb of Rober...,39-story tower with 336 apartments,"25.801424299938954, -80.19525048627189",NaN
3,"2100 Northwest Miami Court, as well as 2101 an...","Miami Court Holdings, led by Joseph Cohen","19-story, 401-unit apartment tower","25.797210654548568, -80.19636345999774",NaN
4,"520, 524, 530, 540, 550, 570, 580 and 590 Nort...",David Sedaghati’s Ultimate Equity,A 25-story complex with 996 apartments and 693...,"25.800676430497518, -80.20452240321376",NaN
5,2601 Northwest Sixth Avenue,Clara Homes,An 18-story to 20-story building with about 15...,"25.80109443927699, -80.2050117352153",NaN
6,"3655 Southwest 22nd Street, Miami",RK Centers (Raanan Katz),"An 8.1-acre site approved for a 1,050-unit Liv...","25.751043987191785, -80.25424939185625",https://therealdeal.com/miami/2025/05/07/raana...


In [6]:
%store -r map_box_api_key

In [ ]:
import folium
from branca.element import Template, MacroElement

# ——— Your title & caption HTML ———
title_html = f'''
  <h3 align="center" style="font-size:16px"><b>Recent {county} County Condo Sales</b></h3>
'''
caption_html = f'''
  <p align="center" style="font-size:13px"><i>{week_starting} — {week_ending}</i></p>
'''

# ——— Base map ———
m = folium.Map(
    location=df[['LATITUDE','LONGITUDE']].mean().to_list(),
    zoom_start=10.5,
    tiles=None
)

# ——— Tile layers ———
folium.TileLayer('OpenStreetMap', name='OSM', control=False).add_to(m)
folium.TileLayer(
    tiles=(
        "https://api.mapbox.com/styles/v1/"
        "mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x"
        f"?access_token={map_box_api_key}"
    ),
    attr="Mapbox", name="Streets", control=False
).add_to(m)

# ——— Add your markers ———
for _, row in df_top_ten.iterrows():
    lat, lon = row['LATITUDE'], row['LONGITUDE']
    popup = folium.Popup(popup_html(row), max_width=400)

    if row['COLOR'] == 'blue':
        # Regular sale: blue circle
        folium.CircleMarker(
            location=(lat, lon),
            radius=7,
            fill=True,
            fill_color='blue',
            fill_opacity=0.5,
            color=None,
            popup=popup
        ).add_to(m)
    else:
        # Top sale: gold star
        folium.Marker(
            location=(lat, lon),
            popup=popup,
            icon=folium.DivIcon(
                html="""
                <div style="font-size:24px; line-height:24px; color:gold;">
                  &#9733;
                </div>
                """
            )
        ).add_to(m)

# ——— Layer control, title & caption ———
folium.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

# ——— Optional legend via branca ———
legend_html = """
{% macro html(this, kwargs) %}
  <div style="
      position: fixed;
      bottom: 50px; left: 50px;
      background: white; padding: 8px;
      border: 2px solid grey;
      font-size: 12px;
      z-index: 9999;
  ">
    <div style="display:inline-block; font-size:18px; color:gold; margin-right:6px;">&#9733;</div> Top Sale<br>
    <i style="background: blue; width: 10px; height: 10px;
       display: inline-block; margin-right: 6px;"></i> All other sales
  </div>
{% endmacro %}
"""
legend = MacroElement()
legend._template = Template(legend_html)
m.get_root().add_child(legend)

# ——— Display ———
m
